In [65]:
!pip install mlxtend


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import pandas as pd

# Load data from local CSV files
orders = pd.read_csv('Machine learning/philkiiru_orders.csv')
trees = pd.read_csv('Machine learning/treez.csv.csv', low_memory=False)
print('Orders preview:')
print(orders.head())
print('\nTrees preview:')
print(trees.head())

Orders preview:
   ORDER_ID  BUYER_ID  TOTAL_PRICE ORDER_STATUS           ORDER_DATE  \
0         6         6       6600.0      PENDING  2025-10-02 05:02:38   
1         7         6      10120.0    CANCELLED  2025-10-02 05:10:31   
2        23         6       1540.0      PENDING  2025-11-25 06:08:58   
3        39         6       3300.0      PENDING  2025-11-25 06:15:47   
4        40         6       2850.0      PENDING  2025-11-25 06:16:04   

         COMMON_NAME  
0   Honeylocust tree  
1   Honeylocust tree  
2   Honeylocust tree  
3   Honeylocust tree  
4   Honeylocust tree  

Trees preview:
  most_recent_observation most_recent_observation_type  common_name  \
0               6/17/2020               inventory_date       Locust   
1              06/09/2020               inventory_date          Oak   
2              06/03/2020               inventory_date          Ash   
3              06/03/2020               inventory_date  Scotch pine   
4               5/29/2020               in

In [67]:
import re


def normalize_name(s):
	if pd.isna(s): return ""
	return re.sub(r"[^\w\s]", "", str(s).lower().strip())


kaggle_common_col = [c for c in trees.columns if "common" in c.lower()][0]


trees["common_norm"] = trees[kaggle_common_col].apply(normalize_name)
orders["common_norm"] = orders["COMMON_NAME"].apply(normalize_name)


# Add any missing trees from orders into trees DataFrame (with placeholder info)
missing_trees = set(orders['common_norm']) - set(trees['common_norm'])
print(f"Missing trees in trees DataFrame: {missing_trees}")
if missing_trees:
	# Add a row for each missing tree with placeholder values for other columns
	for tree in missing_trees:
		placeholder = {col: '' for col in trees.columns}
		placeholder['common_norm'] = tree
		trees = pd.concat([trees, pd.DataFrame([placeholder])], ignore_index=True)
	print(f"Added {len(missing_trees)} missing trees to trees DataFrame.")
else:
	print("No missing trees to add.")


valid_names = set(trees["common_norm"])
orders = orders[orders["common_norm"].isin(valid_names)]


print("✅ Filtered orders:", len(orders))
print(orders.head())

Missing trees in trees DataFrame: {'cypress', 'jacaranda', 'pinus', 'acacia', 'eucalyptus', 'honeylocust tree', 'flame tree', 'grevillea'}
Added 8 missing trees to trees DataFrame.
✅ Filtered orders: 294
   ORDER_ID  BUYER_ID  TOTAL_PRICE ORDER_STATUS           ORDER_DATE  \
0         6         6       6600.0      PENDING  2025-10-02 05:02:38   
1         7         6      10120.0    CANCELLED  2025-10-02 05:10:31   
2        23         6       1540.0      PENDING  2025-11-25 06:08:58   
3        39         6       3300.0      PENDING  2025-11-25 06:15:47   
4        40         6       2850.0      PENDING  2025-11-25 06:16:04   

         COMMON_NAME       common_norm  
0   Honeylocust tree  honeylocust tree  
1   Honeylocust tree  honeylocust tree  
2   Honeylocust tree  honeylocust tree  
3   Honeylocust tree  honeylocust tree  
4   Honeylocust tree  honeylocust tree  


In [68]:
transactions = orders.groupby("ORDER_ID")["common_norm"].apply(list).tolist()
print("Sample transaction baskets:")
for t in transactions[:5]:
    print(t)

Sample transaction baskets:
['austrian pine', 'deodar cedar', 'grevillea', 'london plane', 'texas red oak']
['crabapple', 'cypress', 'desert willow', 'siberian elm']
['crabapple']
['honeylocust tree', 'austrian pine', 'bur oak', 'cottonwood', 'crabapple', 'deodar cedar', 'flame tree', 'siberian elm', 'texas red oak']
['honeylocust tree', 'austrian pine', 'bur oak', 'crabapple', 'deodar cedar', 'desert willow', 'flame tree', 'grevillea', 'london plane', 'siberian elm', 'texas red oak']


In [75]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_enc = pd.DataFrame(te_ary, columns=te.columns_)

# Use lower thresholds for sparse/small datasets
min_support = 0.01  # Lowered for more results
min_confidence = 0.1

frequent_itemsets = apriori(df_enc, min_support=min_support, use_colnames=True)
if frequent_itemsets.empty:
    print(f"⚠️ No frequent itemsets found with min_support={min_support}. Try lowering it further or check your data.")
    rules = pd.DataFrame()
    strong_rules = pd.DataFrame()
else:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    if rules.empty:
        print(f"⚠️ No association rules found with min_confidence={min_confidence}. Try lowering it further.")
        strong_rules = pd.DataFrame()
    else:
        strong_rules = rules[(rules["confidence"] >= 0.7) & (rules["lift"] > 1)]
        print("✅ Strong rules found:", len(strong_rules))
        print(strong_rules[["antecedents","consequents","support","confidence","lift"]].head(10))

✅ Strong rules found: 3722
                       antecedents      consequents  support  confidence  \
166               (ponderosa pine)    (scotch pine)    0.024        0.75   
172       (austrian pine, bur oak)      (crabapple)    0.024        1.00   
174           (bur oak, crabapple)  (austrian pine)    0.024        0.75   
184    (flame tree, austrian pine)        (bur oak)    0.016        1.00   
185          (flame tree, bur oak)  (austrian pine)    0.016        1.00   
192    (bur oak, honeylocust tree)  (austrian pine)    0.016        1.00   
196  (siberian elm, austrian pine)        (bur oak)    0.016        1.00   
197        (siberian elm, bur oak)  (austrian pine)    0.016        1.00   
210      (deodar cedar, crabapple)  (austrian pine)    0.016        1.00   
214    (flame tree, austrian pine)      (crabapple)    0.016        1.00   

          lift  
166  10.416667  
172  10.416667  
174   6.696429  
184  10.416667  
185   8.928571  
192   8.928571  
196  10.416667  


In [76]:
def recommend(tree, rules_df=strong_rules, top_n=5):
    recs = rules_df[rules_df['antecedents'].apply(lambda x: tree in x)]
    if recs.empty:
        print(f"No recommendations for {tree}")
        return
    recs = recs.sort_values(by=["confidence","lift"], ascending=False)
    for _, r in recs.head(top_n).iterrows():
        print(f"💡 If buyer gets {list(r['antecedents'])} → recommend {list(r['consequents'])} "
              f"(conf={r['confidence']:.2f}, lift={r['lift']:.2f})")

# for instance
recommend("crabapple")


💡 If buyer gets ['deodar cedar', 'crabapple'] → recommend ['flame tree', 'austrian pine'] (conf=1.00, lift=62.50)
💡 If buyer gets ['deodar cedar', 'crabapple'] → recommend ['austrian pine', 'siberian elm'] (conf=1.00, lift=62.50)
💡 If buyer gets ['deodar cedar', 'crabapple'] → recommend ['flame tree', 'bur oak'] (conf=1.00, lift=62.50)
💡 If buyer gets ['deodar cedar', 'crabapple'] → recommend ['bur oak', 'honeylocust tree'] (conf=1.00, lift=62.50)
💡 If buyer gets ['deodar cedar', 'crabapple'] → recommend ['siberian elm', 'bur oak'] (conf=1.00, lift=62.50)


In [91]:
# Export recommendations to CSV for use in your PHP project
export_cols = ["antecedents", "consequents", "support", "confidence", "lift"]
rules_to_export = rules.copy()
rules_to_export["antecedents"] = rules_to_export["antecedents"].apply(lambda x: ', '.join(list(x)))
rules_to_export["consequents"] = rules_to_export["consequents"].apply(lambda x: ', '.join(list(x)))
rules_to_export[export_cols].to_csv("ml/recommendations.csv", index=False)
print("Exported recommendations to ml/recommendations.csv")

Exported recommendations to ml/recommendations.csv


In [92]:
# Add image path for each recommended tree (consequent) and export to CSV
import os

# List all files in uploads directory
uploads_dir = 'uploads'
upload_files = os.listdir(uploads_dir)

# Helper to find best image match for a tree name
import difflib
def find_best_image(tree_name, files):
    tree_name = tree_name.lower().replace(' ', '')
    candidates = [f for f in files if tree_name in f.lower().replace(' ', '')]
    if candidates:
        return os.path.join(uploads_dir, candidates[0])
    # fallback: closest match
    match = difflib.get_close_matches(tree_name, [f.lower().replace(' ', '') for f in files], n=1)
    if match:
        idx = [f.lower().replace(' ', '') for f in files].index(match[0])
        return os.path.join(uploads_dir, files[idx])
    return ''

rules_to_export = rules.copy()
rules_to_export["antecedents"] = rules_to_export["antecedents"].apply(lambda x: ', '.join(list(x)))
rules_to_export["consequents"] = rules_to_export["consequents"].apply(lambda x: ', '.join(list(x)))
rules_to_export["consequent_image"] = rules_to_export["consequents"].apply(lambda x: find_best_image(x, upload_files))
export_cols = ["antecedents", "consequents", "consequent_image", "support", "confidence", "lift"]
rules_to_export[export_cols].to_csv("ml/recommendations.csv", index=False)
print("Exported recommendations with images to ml/recommendations.csv")

Exported recommendations with images to ml/recommendations.csv
